In [2]:
import pandas as pd
import numpy as np

## EDA Problem 1
Meteorologia

Generacion_fotovoltaica

Consumo_fotovoltaica

In [3]:
meteo = pd.read_csv("datasets/Meteorologia.csv", sep=",")
genfot = pd.read_csv("datasets/Generacion_fotovoltaica.csv", sep=",")
confot = pd.read_csv("datasets/Consumo_fotovoltaica.csv", sep=",")

In [4]:
genfot.head()

,FECHA,TOTAL_KWH_ENERGIA
0,2023-07-24 11:00:00,NaN
1,2023-07-24 12:00:00,115.66
2,2023-07-24 13:00:00,129.98
3,2023-07-24 14:00:00,134.35
4,2023-07-24 15:00:00,133.90


In [15]:
genfot.tail()

,FECHA,TOTAL_KWH_ENERGIA
9703,2024-08-31 19:00:00,NaN
9704,2024-08-31 20:00:00,NaN
9705,2024-08-31 21:00:00,NaN
9706,2024-08-31 22:00:00,NaN
9707,2024-08-31 23:00:00,NaN


In [11]:
(genfot["TOTAL_KWH_ENERGIA"].isna().sum())/len(genfot["TOTAL_KWH_ENERGIA"])

np.float64(0.11990111248454882)

In [12]:
len(genfot["TOTAL_KWH_ENERGIA"])

9708

In [14]:
genfot.isna()

,FECHA,TOTAL_KWH_ENERGIA
0,False,True
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
9703,False,True
9704,False,True
9705,False,True
9706,False,True
